In [4]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter


from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display

 
import sklearn as sk
import itertools

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time
import copy


In [5]:
attri=["Year","Month","DayofMonth","DayOfWeek","DepTime","CRSDepTime","ArrTime","CRSArrTime", 8
"UniqueCarrier","FlightNum","TailNum","ActualElapsedTime","CRSElapsedTime","AirTime", 6
"ArrDelay","DepDelay","Origin","Dest","Distance","TaxiIn","TaxiOut","Cancelled","CancellationCode","Diverted", 10
"CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay"] 5
print len(attri)

1987,10,14,3,741,730,912,849,
PS,1451,NA,91,79,NA,
23,11,SAN,SFO,447,NA,NA,0,NA,0,
NA,NA,NA,NA,NA

29


In [9]:
#BT data loading

def na_filter(r):
    if str(r[14]) == "NA" \
    or str(r[0]) == "NA" or str(r[1]) == "NA" or str(r[2]) == "NA"  \
    or str(r[3]) == "NA" or str(r[8]) == "NA" or str(r[9]) == "NA" or str(r[16]) == "NA" \
        or str(r[17]) == "NA":
        return False
    else:
        return True


tdta_rdd= sc.textFile("file:///home/tguo/data/tian-real/"+ "1987"  +".csv")
tdta_rdd.cache()

for i in range(1987, 1988):
    
    raw_rdd = sc.textFile("file:///home/tguo/data/tian-real/"+ str(i)  +".csv")
    raw_rdd.cache()
    header = raw_rdd.first()
    raw_rdd = raw_rdd.filter(lambda x:x !=header)    #filter out header
    
    raw_rdd = raw_rdd.map(lambda line: line.split(",")).filter(lambda line: na_filter(line) )
#     print raw_rdd.count()    
    
 
    
    dta_rdd = raw_rdd.map(lambda r: (float(r[14]),
                                    int( float(r[0])), 
                                                                 int(float(r[1])),int(float(r[2])),
                                                                 int(float(r[3])),(str(r[8])),
                                                                 int(float(r[9])),(str(r[16])),
                                                                 (str(r[17])) )
                                                                  )
    if i == 1987:
        tdta_rdd= dta_rdd
    else:
        tdta_rdd = tdta_rdd.union(dta_rdd)
    
    print "data file: ",i
    print dta_rdd.first()
    print dta_rdd.count()
    print tdta_rdd.count()

# feature value reset
feature_valset=[]
feature_valmap=[]
feature_num = 8

print "feature value set:"
for i in range(0, feature_num):
    feature_valset.append( [] )
    feature_valmap.append( {} )
    
    feature_valset[i]= tdta_rdd.map(lambda line: line[ i+1  ]).distinct().collect()
    tmpcnt=len(feature_valset[i])
    feature_valmap[i]= dict(zip( feature_valset[i], range(0,tmpcnt)))
    
    print len(feature_valset[i])

print "feature value re-set:"
# feature value reset
def instance_value_reset(line, feature_num ):
    tmpline=[]
    tmpline.append( line[0] )
    for i in range(0,feature_num):
        tmpline.append(  feature_valmap[i][line[i+1]] )
    return tmpline
    
dta_reset_rdd = tdta_rdd.map(lambda line: instance_value_reset(line,feature_num))    
dta_reset_rdd.cache()

print tdta_rdd.first()
print dta_reset_rdd.first()
print dta_reset_rdd.count()

data file:  1987
(23.0, 1987, 10, 14, 3, 'PS', 1451, 'SAN', 'SFO')
1288326
1288326
feature value set:
1
3
31
7
14
2161
237
237
feature value re-set:
(23.0, 1987, 10, 14, 3, 'PS', 1451, 'SAN', 'SFO')
[23.0, 0, 0, 20, 2, 10, 154, 11, 150]
1288326


In [11]:
dta_reset_rdd.map(lambda line: str(line[0])+ "," + str(line[1])+ 
                              "," + str(line[2])+ "," + str(line[3])+ 
                              "," + str(line[4])+ "," + str(line[5])+
                              "," + str(line[6])+ "," + str(line[7])+ ","+ str(line[8])).saveAsTextFile("hdfs://computer61.ant-net/user/tguo/airline_data_1year.csv")

In [13]:
dta_rdd = sc.textFile("hdfs://computer61.ant-net/user/tguo/airline_data_1year.csv")
print dta_rdd.first()

23.0,0,0,20,2,10,154,11,150
